In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully 

In [11]:
!python -m spacy download en_core_web_sm


2023-12-09 16:37:54.395881: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 16:37:54.395969: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 16:37:54.396015: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 16:37:55.651272: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})

In [12]:
!pip install --upgrade spacy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.6/920.6 kB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.1
    Uninstalling spacy-3.2.1:
      Successfully uninstalled spacy-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.2.0 requires spacy<3.3.0,>=3.2.0, but you have spacy 3.7.2 which is incompatible.


In [15]:
import spacy
import requests

def download_dataset(url):
    response = requests.get(url)
    return response.text

# Download and load the pre-trained SpaCy model
nlp = spacy.load("en_core_web_sm")

# Example usage
dataset_url = "https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttrain.bio"
train_data_raw = download_dataset(dataset_url)

# Process the dataset to create training data
train_data = []
for line in train_data_raw.splitlines():
    if line and "start" not in line:  # Skip the header line
        parts = line.split("\t")
        text = parts[0]
        entities_str = parts[1]

        # Extracting entities as a list of tuples (start, end, label)
        entities = []
        for entity_str in entities_str.split(","):
            entity_parts = entity_str.split()
            if len(entity_parts) >= 4:
                start = int(entity_parts[1])
                end = int(entity_parts[2])
                label = entity_parts[3]
                entities.append((start, end, label))

        train_data.append((text, {"entities": entities}))

# Add NER pipe to the existing SpaCy model
ner = nlp.get_pipe("ner")

# Add your custom entities to the NER pipeline
for _, annotations in train_data:
    for ent in annotations.get("entities", []):
        ner.add_label(ent[2])

# Disable other pipes during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    # Batch Training
    batch_size = 32
    for i in range(0, len(train_data), batch_size):
        batch = train_data[i:i + batch_size]
        examples = [spacy.training.Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in batch]
        nlp.update(examples, drop=0.5, losses={})

# Save the trained model to disk (optional)
nlp.to_disk("ner_model")

# Example usage of the trained model
test_text = "Apple Inc. is a company based in Cupertino."
doc = nlp(test_text)
entities = [{"text": ent.text, "start": ent.start_char, "end": ent.end_char, "label": ent.label_}
            for ent in doc.ents]

# Print the results
print("Named Entities:")
for entity in entities:
    print(entity)


Named Entities:
{'text': 'Cupertino', 'start': 33, 'end': 42, 'label': 'GPE'}
